# Output Formatters Tutorial

---

### To Get the Most Out of this Tutorial
You may read more about the `output_formatter` plugin type <br/>
on the accompanying slides [hosted at this link](https://docs.google.com/presentation/d/1wFbrP4RBdwf6Hx30adD9DN4jNZm1f0hVaHxV--encS0/edit?usp=sharing).

## System requirements  
- **CPU:** 1 CPU  
- **RAM:** 40GB as the notebook is written, but could use more if modified.  
  Reading the entire full-disk ABI image can take up to 100GB.  
- **Disk Space:** 3GB storage space.  
  
***To see the default storage location for your system, run the cell below.***  
To change the default location, change the value of `tmp_root` in the following cell.  

In [ ]:
import tempfile
from pathlib import Path

# Set this to a different path if you would like to use a different location
# `tempfile.gettempdir()` returns a different location on different OS.

# tmp_root = "/use/this/path/instead"
tmp_root = Path(tempfile.gettempdir()) / "geoips_tutorial_tempdirs"

print(f"Notebook temporary storage path: {tmp_root}")

## Important notes  
This notebook downloads approximately 2GB of data and produces another 1GB. It <br />
is stored in the location reported by running the next cell.  
  
This notebook makes an attempt at cleaning up after itself, but it is <br />
recommended that, when done using this notebook, you check to be sure that the <br />
directory reported by the next cell has been deleted.  
  
## Setting up your Environment  
  
**If you don't have an environment already set up, please follow the steps 2 and 3 of the** <br />
**instructions linked [here](https://nrlmmd-geoips.github.io/geoips/getting-started/installing/index.html) based on the architecture of your machine.**  
  
You can quit following the instructions after you've activated your geoips environment via <br />
``conda activate geoips``.  
  
Run the cell below to set up your environment. This will initialize a session-specific <br />
storage directory, add it to the global notebook environment, and add a hook that attempts <br />
to clean up the temporary storage when the notebook is closed.  
  
❗***Important:*** While this notebook makes an effort to clean up after itself, if you <br />
are running this notebook on your own system, ***it is advisable tomanually delete the <br />
temporary directory reported above when you are done using the notebook.***

In [ ]:
import dotenv
from IPython import get_ipython
import os

from geoips.utils import notebook_environment

# Sets up the environment and sets a global variable named `temp_dir`
notebook_environment.setup(tmp_root)

with open("./.env", "w") as env_file:
    env_file.writelines(
        [
            f"GEOIPS_TESTDATA_DIR={get_ipython().user_ns['temp_dir']}/test_data\n",
            f"GEOIPS_OUTDIRS={get_ipython().user_ns['temp_dir']}/outdirs\n",
            f"GEOIPS_PACKAGES_DIR={get_ipython().user_ns['temp_dir']}\n",
            f"GEOIPS_REBUILD_REGISTRIES=True",
            f"MY_PKG_DIR={get_ipython().user_ns['temp_dir']}/cool_plugins\n",
            "MY_PKG_NAME=cool_plugins"
        ],
    )

dotenv.load_dotenv("./.env", override=True)

if not os.path.exists(os.environ["GEOIPS_TESTDATA_DIR"]):
    os.makedirs(os.environ["GEOIPS_TESTDATA_DIR"])
if not os.path.exists(os.environ["GEOIPS_OUTDIRS"]):
    os.makedirs(os.environ["GEOIPS_OUTDIRS"])


In [ ]:
# Initial Setup:
from glob import glob
import xarray as xr
import logging
from datetime import datetime, timezone
import geoips

LOG = logging.getLogger(__name__)

## Importing the GeoIPS Interfaces

Since we are walking through this tutorial as a Python script, 
we need to import every plugin that we plan to use.

We use the convention:
`{interface_type}.get_plugin({plugin_name})` 
to load the plugins at runtime .

In [ ]:
# import every interface type that we are going to use
from geoips.interfaces import readers, algorithms, interpolators, sectors, output_formatters, colormappers, gridline_annotators, feature_annotators

# Since we are reading abi data in a netCDF format, we will use this reader
abi_reader = readers.get_plugin("abi_netcdf")
# Since we will be reading a single channel, we want to use the single_channel algorithm
single_channel_algorithm = algorithms.get_plugin("single_channel")
# We want to use Nearest Neighbor resampling for this dataset, so we will use the interp_nearest interpolator plugin
interp_nearest = interpolators.get_plugin("interp_nearest")
# We want to restrict our output to just the CONUS sector
conus_sector_plugin = sectors.get_plugin("conus")

Since we load these plugins dynamically, it can take some extra work to check
their call signatures.  
We can look more closely at the reader plugin by calling:
(this only works in ipython or jupyter notebooks)

In [ ]:
abi_reader?

Here we can see the call signature and docstring from the plugin.  
This should be enough to help you figure out what arguments to pass.

## Reading the ABI Test Data
  
For this test, we will use the data stored in the `GEOIPS_TESTDATA_DIR` directory.
  
We can access the current GeoIPS environment variables by calling:  
`geoips.filenames.base_paths.PATHS`  
which returns a dictionary with the environment variables.

In [ ]:
GEOIPS_TESTDATA_DIR = geoips.filenames.base_paths.PATHS['GEOIPS_TESTDATA_DIR']
in_fpath = f"{GEOIPS_TESTDATA_DIR}/test_data_abi/data/goes16_20200918_1950/*"
in_fpaths = glob(in_fpath)
success_outputs=[]

In [ ]:
# The conus_sector_plugin contains an area_definition.
# We want to use that area_definition when calling our reader.
conus_sector_area = conus_sector_plugin.area_definition

# First we need to read the abi data for channel 14 Brightness Temperature
# xdict = abi_reader(fnames=in_fpaths, chans=["B14BT"])
xdict = abi_reader(fnames=in_fpaths, area_def=conus_sector_area, chans=["B14BT"])

# Since we passed an area_def for "conus", that is the name of the Xarray Object we want to call.
conus_xobj = xdict["conus"]

# Let's look at what conus_xobj contains within it:
conus_xobj

## Applying the Interpolator
Now we will apply the interpolator. Let's look at the call signature:

In [ ]:
interp_nearest?

In [ ]:
# Now let us apply the Nearest Neighbor interpolation to our data
output_dataset = interp_nearest(conus_sector_area,
                                conus_xobj,
                                None,
                                ["B14BT", "longitude", "latitude"])

# Let's look at our output_dataset
output_dataset

In [ ]:
channel_14_bt = output_dataset["B14BT"].data

# Now let us apply the single channel algorithm to our channel 14 Brightness Temperature data
algorithm_output = single_channel_algorithm([channel_14_bt],
                                            output_data_range=[-90.0, 30.0],
                                            input_units="Kelvin",
                                            output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

# Let's look at this dataset now that we have created it
output_dataset

## NetCDF Output Formatter

To start off, let's try putting our final data into a netcdf file

In [ ]:
ncdf_output_formatter = output_formatters.get_plugin("netcdf_geoips")

timestamp = datetime.strftime(datetime.now(timezone.utc), "%Y%m%d%H%M%S")
GEOIPS_OUTDIRS = geoips.filenames.base_paths.PATHS['GEOIPS_OUTDIRS']
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_xarray_test_{timestamp}.nc"

netcdf_output = ncdf_output_formatter(output_dataset,
                                        ["Infrared"],
                                        [out_fpath])
print(f"NetCDF output located at:  {netcdf_output[0]}")

In [ ]:
ncdf_output_formatter?

In [ ]:
# We need a colormapper to tell matplotlib what colors we want to use
ir_colormapper = colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [ ]:
#---------------#
# imagery_clean #
#---------------#

# We can also create clean imagery
img_clean_output_formatter = output_formatters.get_plugin("imagery_clean")

out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_clean_test_{timestamp}.png"

png_clean_output = img_clean_output_formatter(conus_sector_area,
                                             output_dataset,
                                             "Infrared",
                                             [out_fpath],
                                             mpl_colors_info=ir_color_dict)

png_clean_output

![image](/home/coleman/geoips/outdirs/abi_infrared_clean_test_20250718183819.png)

In [ ]:
# We need to tell matplotlib how we want the resulting plot to appear
# using a feature_annotator and a gridline_annotator.
# This time we will use the default versions of these plugins.
local_feature_annotator = feature_annotators.get_plugin("default")

local_gridline_annotator = gridline_annotators.get_plugin("default")

In [ ]:
#-------------------#
# imagery_annotated #
#-------------------#

# Now that we have all the setup completed, we can call our output formatter.
# Let's start by loading the imagery_annotated plugin
img_ann_output_formatter = output_formatters.get_plugin("imagery_annotated")

formatter_result_dict = {}
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_annotated_test_{timestamp}.png"

# And then we call the plugin
png_annotated_output = img_ann_output_formatter(conus_sector_area,
                                           output_dataset,
                                           "Infrared",
                                           [out_fpath],
                                           mpl_colors_info=ir_color_dict,
                                           feature_annotator=local_feature_annotator,
                                           gridline_annotator=local_gridline_annotator,
                                           output_dict=formatter_result_dict)

# Let's print the path to the image on your disk
png_annotated_output

![image](/home/coleman/geoips/outdirs/abi_infrared_annotated_test_20250718183819.png)

### Clean Up

Run the following segment to clean up the output files we created with this notebook:

In [ ]:
try:
    if netcdf_output:
        Path(netcdf_output[0]).unlink(missing_ok=True)
        print(f"Deleted {netcdf_output[0]}")
except NameError:
    None

try:
    if png_clean_output:
        Path(png_clean_output[0]).unlink(missing_ok=True)
        print(f"Deleted {png_clean_output[0]}")
except NameError:
    None

try:
    if png_annotated_output:
        Path(png_annotated_output[0]).unlink(missing_ok=True)
        print(f"Deleted {png_annotated_output[0]}")
except NameError:
    None